<a href="https://colab.research.google.com/github/dhivyanj/qff25-hthn/blob/main/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U qiskit qiskit-aer qiskit-ibm-runtime qiskit[visualization] qiskit_aer --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00


In [1]:
!pip install qiskit qiskit-aer qiskit-machine-learning
!pip install numpy pandas scikit-learn matplotlib


import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Latest Qiskit imports (no optimizer needed for our approach)
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator, Sampler
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.circuit.library import ZFeatureMap, RealAmplitudes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2


In [ ]:
class QiskitQuantumReservoir:
    def __init__(self, n_qubits=4, shots=1000):
        self.n_qubits = n_qubits
        self.shots = shots
        self.backend = Aer.get_backend('qasm_simulator')
        self.statevector_backend = Aer.get_backend('statevector_simulator')

        # Create fixed random angles for reservoir
        self.reservoir_angles = np.random.uniform(0, 2*np.pi, size=(n_qubits, 3))

    def create_reservoir_circuit(self, input_data):
        """Create quantum reservoir circuit"""
        qc = QuantumCircuit(self.n_qubits)

        # Step 1: Encode input data as rotations
        for i in range(min(len(input_data), self.n_qubits)):
            qc.ry(input_data[i] * np.pi, i)  # Encode to [0, π] range

        # Step 2: Fixed reservoir dynamics
        # Entangling layer
        for i in range(self.n_qubits - 1):
            qc.cx(i, i+1)
        qc.cx(self.n_qubits-1, 0)  # Circular entanglement

        # Fixed rotational gates
        for i in range(self.n_qubits):
            qc.rz(self.reservoir_angles[i, 0], i)
            qc.ry(self.reservoir_angles[i, 1], i)
            qc.rz(self.reservoir_angles[i, 2], i)

        # Another entangling layer
        for i in range(self.n_qubits - 1):
            qc.cx(i, i+1)

        return qc

    def get_quantum_features(self, input_data):
        """Get quantum features from input data"""
        qc = self.create_reservoir_circuit(input_data)

        # Method 1: Use statevector for exact probabilities
        job = execute(qc, self.statevector_backend)
        result = job.result()
        statevector = result.get_statevector()
        probabilities = np.abs(statevector)**2

        # Method 2: Use measurements for expectation values
        qc_measure = qc.copy()
        qc_measure.measure_all()
        job = execute(qc_measure, self.backend, shots=self.shots)
        result = job.result()
        counts = result.get_counts()

        # Get expectation values from counts
        expectation_values = []
        total_shots = sum(counts.values())
        for i in range(self.n_qubits):
            # Calculate <Z_i> = (counts(0) - counts(1)) / total
            count_0 = sum(v for k, v in counts.items() if k[-(i+1)] == '0')
            count_1 = total_shots - count_0
            exp_val = (count_0 - count_1) / total_shots
            expectation_values.append(exp_val)

        # Combine probabilities and expectation values
        quantum_features = np.concatenate([probabilities, expectation_values])
        return quantum_features

    def compute_reservoir_states(self, sequences):
        """Compute reservoir states for all sequences"""
        reservoir_states = []
        for sequence in sequences:
            sequence_features = []
            for timestep in sequence:
                # Normalize timestep data
                normalized_timestep = timestep / np.linalg.norm(timestep) if np.linalg.norm(timestep) > 0 else timestep
                features = self.get_quantum_features(normalized_timestep)
                sequence_features.append(features)
            # Flatten all timestep features for the sequence
            reservoir_states.append(np.concatenate(sequence_features))

        return np.array(reservoir_states)